<a href="https://colab.research.google.com/github/emilfunk/Fundamentals-of-Accelerated-Data-Science-with-RAPIDS/blob/main/Copy_of_ProToken_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/Dreams1de/ProToken/raw/main/fig1.png" height="200" align="middle" style="height:240px">

##ProToken 1.0: Compact and Informative Encoding of Protein 3D Structures

Easy to use protein (complex) structure encoding and decoding using ProToken. For more details, see <a href="#Instructions">bottom</a> of the notebook, checkout the [ProToken-1.0 Model](https://drive.google.com/file/d/1z2X_Ly-HXpDryqJIGtnOCuddQi_eIoHS/view?usp=drive_link), [ProToken-1.0 Code Book](https://drive.google.com/file/d/1PpK8iKcD2OoQvcDs2bnsA9dJ97GnhrNq/view?usp=drive_link) and read our manuscript.

[Xiaohan Lin, Zhenyu Chen, Yanheng Li, Xingyu Lu, Chuanliu Fan, Ziqiang Cao, Shihao Feng, Yi Qin Gao, Jun Zhang. ProTokens: A Machine-Learned Language for Compact and Informative Encoding of Protein 3D Structures.
*biorxiv*, 2023](https://www.biorxiv.org/content/10.1101/2023.11.27.568722v2.abstract)

Report issues or bugs, please contact: fengsh@cpl.ac.cn or jzhang@cpl.ac.cn. For prompt reply, It is recommended to include #ProToken# in your mail title.

Thanks for [Sergey Ovchinnikov's ColabFold Colab Notebook](https://colab.research.google.com/github/sokrypton/ColabFold/blob/main/AlphaFold2.ipynb#scrollTo=kOblAo-xetgx) for reference.

In [ ]:
# ==============================================================================
# Copyright 2024 Changping Laboratory & Peking University. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 (the “License”);
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an “AS IS” BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

In [ ]:
#@title Input protein structure(s), then hit `Runtime` -> `Run all`

# download the package from google drive and prepare the libraries
import os
if not os.path.exists('/content/ProToken-1.0'):
  !gdown --id '1z2X_Ly-HXpDryqJIGtnOCuddQi_eIoHS'
  !gdown --id '1PpK8iKcD2OoQvcDs2bnsA9dJ97GnhrNq'
  !tar -xJf '/content/ProToken-1.0.tar.xz'
  !pip install biopython -q

from google.colab import files
import os
jobname = 'test' #@param {type:"string"}

user_mode = "single_example" #@param ["single_example", "multimer_example","custom_example"]
#@markdown - `single_example` = using example in /content/ProToken-1.0/examples/single (see [notes](#single_example)).
#@markdown - `multimer_example` = using example in /content/ProToken-1.0/examples/multimer (see [notes](#multimer_example)).
#@markdown - `custom_example` = upload and use own protein structure(s) (PDB format **only contains lines start with 'ATOM'**. see [notes](#custom_example)).

task_mode = "single" #@param ["single", "multi"]
#@markdown - `single` = support one single-chain pdb file under the input pdb file directory.
#@markdown - `multi` = support more than one single-chain pdb files under the input pdb file directory.

# check if directory with jobname exists
def check(folder):
  if os.path.exists(folder):
    return False
  else:
    return True
if not check(jobname):
  n = 0
  while not check(f"{jobname}_{n}"): n += 1
  jobname = f"{jobname}_{n}"

# make directory to save results
os.makedirs(jobname, exist_ok=True)

if user_mode == 'custom_example':
  input_pdb_dir = os.path.join(jobname,f"input_pdbs")
  os.makedirs(input_pdb_dir, exist_ok=True)
  uploaded = files.upload()
  for fn in uploaded.keys():
    os.rename(fn,os.path.join(input_pdb_dir,fn))
  pdb_input_dir = input_pdb_dir
  if task_mode == 'multi':
    assert len(list(uploaded.keys())) > 1, 'task_mode should be set to single!~'
  if task_mode == 'single':
    assert len(list(uploaded.keys())) == 1, 'task_mode should be set to multi!~'

if user_mode == 'multimer_example':
  assert task_mode == 'multi', 'task_mode should be set to multi!~'
  pdb_input_dir = '/content/ProToken-1.0/examples/multimer'
if user_mode == 'single_example':
  assert task_mode == 'single', 'task_mode should be set to single!~'
  pdb_input_dir = '/content/ProToken-1.0/examples/single'

pdb_saving_path = os.path.join(jobname, 'reconstructed_protein.pdb')
code_saving_path = os.path.join(jobname, 'protoken_index.pkl')

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1z2X_Ly-HXpDryqJIGtnOCuddQi_eIoHS
From (redirected): https://drive.google.com/uc?id=1z2X_Ly-HXpDryqJIGtnOCuddQi_eIoHS&confirm=t&uuid=436d8c47-c45b-41ee-9a2c-4230a28ebb1e
To: /content/ProToken-1.0.tar.xz
100% 361M/361M [00:07<00:00, 51.2MB/s]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.9 MB/s eta 0:00:00


In [ ]:
#@title 1. Import the libraries
import os, jax
import pickle as pkl
import numpy as np
import tensorflow as tf

protoken_base_dir = '/content/ProToken-1.0'
import sys
sys.path.append(protoken_base_dir)
from data_process.preprocess import save_pdb_from_aux, protoken_encoder_preprocess, protoken_decoder_preprocess, init_protoken_model
from data_process.preprocess import protoken_encoder_input_features, protoken_decoder_input_features

In [ ]:
#@title 2. Prepare the encoder's inputs

encoder_inputs, encoder_aux, seq_len = protoken_encoder_preprocess(pdb_input_dir, task_mode=task_mode)
for k, v in zip(protoken_encoder_input_features, encoder_inputs):
    print(k, v.shape)

if task_mode == 'multi':
  # multimer auxiliary information
  print('\nHere is the brief information for multi-chain complexes:')
  for k, v in encoder_aux['chain_length_info'].items():
    print(f'chain {k}: ', v)

Found 1 pdb files in the input directory.
seq_mask (512,)
residue_index (512,)
backbone_atom_masks (512, 37)
backbone_atom_positions (512, 37, 3)
ca_pos (512, 3)
backbone_affine_tensor (512, 7)
torsion_angles_sin_cos (512, 6)
torsion_angles_mask (512, 3)


In [ ]:
#@title 3. Warm up the encoder and decoder

#@markdown We have 3 models for different total sequence lengths ranging from `0-512`, `512-1024`, `1024-2048`.

#@markdown You can choose the model based on the sequence length of your protein. Currently, T4 GPU supports the sequence length up to 1024.

#@markdown Once the total sequence length is beyond the current model's length coverage, you have to `reinitialize the model`.

#@markdown Have fun!

model = init_protoken_model(seq_len, '/content/ProToken-1.0')

Found GPU, will use GPU for prediction


In [ ]:
#@title 4. Encode the protein structure and get the ProToken Index

encoder_results = model.encoder(*encoder_inputs)

if task_mode == 'multi':
  protoken_index_ = np.asarray([encoder_results["protoken_index"][p] for p in range(encoder_aux['seq_mask'].shape[0]) \
                                if encoder_aux['seq_mask'][p]])
  protoken_index_multimer = [protoken_index_[v['start_idx']:v['start_idx']+v['seq_len']] for k, v in encoder_aux['chain_length_info'].items()]
  for k in encoder_aux['chain_length_info'].keys():
      print('PDB ID: ', encoder_aux['chain_length_info'][k]['pdb_name'])
      print(f'Chain Length: ', encoder_aux['chain_length_info'][k]['seq_len'])
      print(f'ProToken Index: {protoken_index_multimer[k].shape}\n{protoken_index_multimer[k]}')
      encoder_aux['chain_length_info'][k]['protoken_index'] = protoken_index_multimer[k]
  with open(code_saving_path, 'wb') as f:
      pkl.dump(encoder_aux['chain_length_info'], f)
else:
  protoken_index = np.asarray([encoder_results["protoken_index"][p] for p in range(encoder_aux['seq_mask'].shape[0]) \
                                if encoder_aux['seq_mask'][p]])
  print(f'ProToken Index: {protoken_index.shape}\n{protoken_index}')
  with open(code_saving_path, 'wb') as f:
      pkl.dump(protoken_index, f)

ProToken Index: (193,)
[258 384 416 294 324 454 324 227 127 104 342 100 373 381  92 215 487 403
  92 250 509 324 240 177 256 472 384  74 228 471  24 241 329 202 369 132
 458 487  47 333 151 267 231 483 133  51  28 132  32   0 362  78 493 220
  24  12 196 364 337 210 358 439 367 161 293 216 450 110 106 266 257 473
 495 291  46 503  92 328 214  48 384 360 146 266 476  50 297 185 241  50
  34 362 241 485 163 237 304  27 419 299  72  42 293 329 430  76 315 152
 481 268 315 123 361  59 194 262 372 248 130 268 425 109 256 118 386 264
 393 305 347 190 411 403 106 407 446  14  38 487 161 342 190 254  42 334
  49 125 187 466 143 457 324 439 109 161 456 163  30 161 415 440 151 170
 291 395 274  42 457 246  25  42 224 315 442 471 349 303 442 202 451 261
  38 272 165 230 466 168 434 247 450 411  52  95 264]


In [ ]:
#@title 5. Prepare the decoder's inputs

if task_mode == 'multi':
  # Multimer ProToken decoder's inputs should be a list of ProToken index and ProToken index should be in np.ndarray format.
  decoder_inputs = protoken_decoder_preprocess(protoken_index_multimer, task_mode=task_mode)
  for k, v in zip(protoken_decoder_input_features, decoder_inputs):
      print(f'{k}: {v.shape}')
else:
  # Single chain ProToken decoder's inputs should be a array of ProToken indexes in the np.ndarray format.
  decoder_inputs = protoken_decoder_preprocess(protoken_index, task_mode=task_mode)
  for k, v in zip(protoken_decoder_input_features, decoder_inputs):
      print(f'{k}: {v.shape}')

protoken_index: (512,)
protoken_mask: (512,)
residue_index: (512,)


In [ ]:
#@title 6. Decode the ProToken Index and get the reconstructed protein structure

decoder_results = model.decoder(*decoder_inputs)
reconstructed_atom_positions = np.asarray(decoder_results['reconstructed_atom_positions'])

In [ ]:
#@title 7. Compare the original and reconstructed protein structures

from data_process.preprocess import lddt
lDDT = lddt(reconstructed_atom_positions[None, ...][:,:,1,:],
            encoder_aux['backbone_atom_positions'][None, ...][:,:,1,:],
            encoder_aux['seq_mask'][None,...,None], per_residue=True)[0]
print(f"Average lDDT: {np.mean(lDDT[:np.sum(encoder_aux['seq_mask'])])}")

Average lDDT: 0.9680577494332036


In [ ]:
#@title 8. Save the reconstructed protein structure

partial_aux = {"aatype": encoder_aux["aatype"].astype(np.int32),
               "residue_index": decoder_inputs[-1].astype(np.int32)+1,
               "atom_positions": reconstructed_atom_positions.astype(np.float32),
               "atom_mask": encoder_aux["backbone_atom_masks"].astype(np.float32),
               "plddt": lDDT.astype(np.float32)}
save_pdb_from_aux(partial_aux, pdb_saving_path)

# if you want to save the protein without encoder_aux,
# use the following code to save the protein
# aatype_all_gly = np.asarray(decoder_inputs[1]).astype(np.int32)*7
# backbone_atom_mask = np.repeat(np.asarray([1,1,1,0,1]+[0]*32)[None,...], aatype_all_gly.shape[0], axis=0).astype(np.float32)*decoder_inputs[1][..., None]
# plddt = np.ones_like(aatype_all_gly).astype(np.float32)*99.99
# partial_aux = {"aatype": aatype_all_gly,
#                "residue_index": decoder_inputs[-1].astype(np.int32)+1,
#                "atom_positions": reconstructed_atom_positions.astype(np.float32),
#                "atom_mask": backbone_atom_mask,
#                "plddt": plddt}
# save_pdb_from_aux(partial_aux, pdb_saving_path)

In [ ]:
#@title 9. Conclusion

print(f'PDB saved at: {pdb_saving_path}')
print(f'ProTokens saved at: {code_saving_path}')
print('Average lDDT:', round(np.mean(lDDT[:np.sum(encoder_aux['seq_mask'])]), 3), 'Seq_Len:', seq_len)
print(f'Job finished!\n')

PDB saved at: test/reconstructed_protein.pdb
ProTokens saved at: test/protoken_index.pkl
Average lDDT: 0.968 Seq_Len: 193
Job finished!



# Instructions <a name="Instructions"></a>

#### <u>Quick start</u>
1. Paste your protein structure(s) in the input field or using the examples provided in the google drive.
2. Press "Runtime" -> "Run all".
3. The pipeline consists of 9 steps. The currently running step is indicated by a circle with a stop sign next to it.

#### <u>Single-Chain Example</u> <a name="single_example"></a>

**Example**: T1024-D1.pdb from [CASP14](https://predictioncenter.org/casp14/index.cgi).

This example involves a single-chain protein structure sourced from CASP14, specifically designed to illustrate the model's application on individual protein chains.

#### <u>Multimer Example</u> <a name="multimer_example"></a>

**Example**: 7W51_A.pdb & 7W51_B.pdb.

These multimer examples were released on October 26, 2022, and are available in the [RCSB Protein Data Bank](https://www.rcsb.org/structure/7W51). The files include structures for two separate chains within a multimeric protein.

#### <u>Custom Examples</u> <a name="custom_example"></a>

Users are encouraged to upload their own PDB files online to test the model. For optimal results, please adhere to the following guidelines:

**Important Guidelines:**
1. Ensure that each PDB file contains the protein structure of only one chain.
2. The PDB file should only contain lines that start with `ATOM`. Lines starting with `HETATM` or other prefixes may adversely affect the model's performance.


#### <u>Local Test</u>

You can download the model and test it on your local devices through [ProToken-1.0 Model](https://drive.google.com/file/d/1z2X_Ly-HXpDryqJIGtnOCuddQi_eIoHS/view?usp=drive_link).


#### <u>ProToken-1.0 Code Book</u>
We also release the [ProToken-1.0 Code Book](https://drive.google.com/file/d/1PpK8iKcD2OoQvcDs2bnsA9dJ97GnhrNq/view?usp=drive_link), which contains a 32-dimensional embedding for each code (512 codes in total).